In [4]:
import wave
import numpy as np
import os
from pydub import AudioSegment

Now we define a function that check the imported cover audio if it has '.wav' extension. If not, we convert it using PyDub.

In [ ]:
def convertToWav(inputFile):
    if not os.path.exists(inputFile):
        print(f"Error: The input file '{inputFile}' does not exist.")
        return None
    _, ext = os.path.splitext(inputFile)
    if ext.lower() == '.wav':
        print(f"The file '{inputFile}' is already in WAV format.")
        return inputFile
    outputFile = os.path.splitext(inputFile)[0] + '.wav'
    try:
        audio = AudioSegment.from_file(inputFile)
        audio.export(output_file, format="wav")
        print(f"Audio successfully converted to WAV format: {outputFile}")
        return outputFile
    except Exception as e:
        print(f"An error occurred during conversion: {e}")
        return None

We do the same function on the message audio.